# Homework 1: Locally Sensitive Hashing
Locality Sensitive Hashing (LSH) is a technique used in computer science to solve the approximate or exact Near Neighbor Search in high-dimensional spaces. It is used to find similar items in a large dataset by hashing input items so that similar items map to the same "buckets" with high probability. LSH is commonly used in recommendation systems, image and audio recognition, and data mining.

In this particular notebook we will implement a simplified version of the LSH algorithm for to compare texts and find how similar are. We will implement 4 classes which will help us to compute how similar 2 texts are. Those classes are: Shingling, CompareSets, MinHasing and CompareSignatures.

## Dataset
As a dataset, we have used the following texts:
 - Lorem Ipsum with 5 paragraphs (https://www.lipsum.com/)\[1.txt\]
 - Lorem Ipsum with 7 paragraphs (https://www.lipsum.com/)\[2.txt\]
 - Quijote de la Mancha by Miguel de Cervantes (https://www.gutenberg.org/cache/epub/60884/pg60884.txt)\[3.txt\]
 - The Adventures of Sherlock Holmes by Arthur Conan Doyle (https://www.gutenberg.org/cache/epub/1661/pg1661.txt)\[4.txt\]
 - The picture of Dorian Gray by Oscar Wilde (https://www.gutenberg.org/cache/epub/174/pg174.txt)\[5.txt\]
 - Beyond good and evil by Friedrich Wilheim Nietzsche (https://www.gutenberg.org/cache/epub/4363/pg4363.txt)\[6.txt\]

This dataset has been selected like this, so it has two text which should be fairly similar (1 & 2), a text which should be fairly different (3) and three texts which should have something in common even though they may be different (4, 5 & 6).

In this notebook we have implemented 4 classes which we will help us to compute the similarity of 2 texts. Those classes are: Shingling, CompareSets, MinHashing and CompareSignatures.

In [1]:
import os

# define the path to the dataset folder
dataset_path: str = "dataset"

# create an empty list to store the texts
texts: list[str] = []

# loop through the files in the dataset folder
for filename in os.listdir(dataset_path):
    # check if the file is a text file
    if filename.endswith(".txt"):
        # open the file and read its contents
        with open(os.path.join(dataset_path, filename), "r") as f:
            text = f.read()
            texts.append(text)

# print the texts
for i, text in enumerate(texts):
    print(f"Text {i+1}:")
    print(text)
    print("-"*132)


Text 1:
HAPTER I.


The studio was filled with the rich odour of roses, and when the light
summer wind stirred amidst the trees of the garden, there came through
the open door the heavy scent of the lilac, or the more delicate
perfume of the pink-flowering thorn.

From the corner of the divan of Persian saddle-bags on which he was
lying, smoking, as was his custom, innumerable cigarettes, Lord Henry
Wotton could just catch the gleam of the honey-sweet and honey-coloured
blossoms of a laburnum, whose tremulous branches seemed hardly able to
bear the burden of a beauty so flamelike as theirs; and now and then
the fantastic shadows of birds in flight flitted across the long
tussore-silk curtains that were stretched in front of the huge window,
producing a kind of momentary Japanese effect, and making him think of
those pallid, jade-faced painters of Tokyo who, through the medium of
an art that is necessarily immobile, seek to convey the sense of
swiftness and motion. The sullen murmur of 

## Shingling
Shingling is a technique used in text analysis to represent a document as a set of overlapping subsequences of fixed length k, called k-shingles. To compute shingles, we slide a window of size k over the document and extract the k-length substrings that fall within the window. We then store these substrings as a set, which represents the shingles of the document.

In [2]:
class Shingling():
    
    def __init__(self, k: int):
        self.k: int = k
        
    def get_shingles(self, document: str) -> set:
        """
        This method constructs k-shingles of a given length k from a given document.
        """
        shingles: set = set()
        for i in range(len(document) - self.k + 1):
            shingle: str = document[i:i+self.k]
            shingles.add(shingle)
        return shingles
    
    def get_hashed_shingles(self, document: str):
        """
        This method computes a hash value for each unique shingle and represents the document in the form of an ordered set of its hashed k-shingles.
        """
        shingles: set = self.get_shingles(document)
        hashed_shingles: list[int] = [hash(shingle) for shingle in shingles]
        hashed_shingles.sort()
        return hashed_shingles

As an example to show what is shingling, we will use the following text:
> "The quick brown fox jumps over the lazy dog"

In [3]:
text = "The quick brown fox jumps over the lazy dog"
shingling: Shingling = Shingling(k=5)
shingles: list[str] = shingling.get_shingles(text)
print(shingles)
shingles_hash: list[int] = shingling.get_hashed_shingles(text)
print("-"*132)
print(shingles_hash)



{' over', 'quick', 'zy do', 'k bro', ' quic', 'wn fo', 'e laz', ' brow', 'ox ju', 'he qu', 'ps ov', 'x jum', 'The q', 'mps o', ' lazy', 'n fox', 'over ', 'rown ', 'r the', 'er th', 'jumps', ' fox ', ' the ', 'ver t', 'y dog', 'ck br', 's ove', 'brown', 'e qui', 'uick ', ' jump', 'the l', 'umps ', 'he la', 'fox j', 'lazy ', 'azy d', 'own f', 'ick b'}
------------------------------------------------------------------------------------------------------------------------------------
[-9064736398775172096, -8904891166855052104, -8755722416764248258, -8644393365592591934, -8454997467552267798, -7110935589167731386, -6447411291771312349, -5872618805289031270, -5775454823953375254, -5758183914033748929, -5176786894885922180, -5020946388002439793, -4736853037037541719, -4706544455482330018, -4572808456746687809, -3474089507501848432, -3353223062236414928, -1938159933856344288, -1273841040590754487, -876481457819425505, 69460134785630011, 702688304341401544, 1107644356025528054, 155528038424997

Now we do it with our text. We use a k=9 as we are analyzing text from books instead of emails. In the following lines we are doing exactly the same as we showed in the example above.

In [4]:
shingling: Shingling = Shingling(k=9)

hashed_shingles_list: list[int] = []

for text in texts:
    hashed_shingles: int = shingling.get_hashed_shingles(text)
    hashed_shingles_list.append(hashed_shingles)
    
for i, hashed_shingles in enumerate(hashed_shingles_list):
    print(f"Hashed shingles for Text {i+1}:")
    print(hashed_shingles)
    print("-"*132)


Hashed shingles for Text 1:
[-9212957088048787652, -9208372001943057261, -9208152312950643817, -9193888716364670684, -9181865187913703077, -9180025253164850719, -9145503782676918409, -9141185912057947185, -9140982588690847011, -9140241760328777061, -9135699658296636229, -9133511066971272603, -9123904412956893631, -9120693284689345552, -9111142194553026964, -9108969182431017002, -9103010907689585914, -9099526628776596096, -9098950365545889356, -9097973316641469921, -9085265569543352566, -9081502235059642837, -9062098492197240705, -9045885519221741618, -9041991336835612893, -9041128098066260298, -9041099472280374557, -9032057124140328935, -9028926103790880739, -9023889822534918004, -9023729855546693152, -9023566111025404651, -9014036305460686121, -9008056118263485108, -9003772680271537487, -9000137640099235748, -8996771539924620521, -8991429990239251767, -8991343154237375950, -8986159251546698104, -8982368843876588513, -8960103625944152556, -8959898181123294084, -8958146741186918980, -89

## CompareSets


## MinHashing

## CompareSignatures

Alina 1:05